# Using LangChain middleware with ChatWriter
[Middleware](https://docs.langchain.com/oss/python/langchain/middleware/overview) is a powerful feature in LangChain that lets you **control and customize the behavior of a ChatWriter agent at every step** of its execution.

This guide demonstrates how to apply both **built-in and custom middleware** to an agent. You'll explore features such as PII detection, call limits, human-in-the-loop workflows, prompt transformation, and more. By following this guide, you'll learn how to **monitor agent behavior, modify tool usage, and integrate advanced logic** into your agent workflow.

## Prerequisites
Before getting started, you'll need:
- A Python environment with **LangChain** installed
- Access to a **ChatWriter-compatible model**
- Any tools you plan to integrate with your agent

## Setup
Install LangChain if you haven't already:

In [ ]:
%pip install langchain-writer langchain langchain_core

Next, set the `WRITER_API_KEY` environment variable. We recommend setting it in a `.env` file in the root of your project, but this tutorial will set it in an environment variable if you don't have a `.env` file.

In [ ]:
import getpass
import os
from writerai import Writer
from langchain_writer import ChatWriter


if not os.getenv("WRITER_API_KEY"):
    os.environ["WRITER_API_KEY"] = getpass.getpass("Enter your Writer API key: ")

chat = ChatWriter(model="palmyra-x5")

## Example: Agent with built-in middleware

List of all built-in [middlewares](https://docs.langchain.com/oss/python/langchain/middleware/built-in).

We create an agent that uses ChatWriter and attach middleware that detects PII in inputs/outputs (via `PIIMiddleware`).

In [ ]:
from langchain.agents import create_agent
from langchain_writer import ChatWriter
from langchain.agents.middleware import PIIMiddleware

# Create agent with ChatWriter and PII detection middleware
agent = create_agent(
    model=chat,  # chat model instance
    middleware=[
        PIIMiddleware("email", strategy="redact", apply_to_input=True)  # redact emails in input
    ]
)


In [ ]:

result = agent.invoke({"messages": [{"role": "user", "content": "extract the email from the next message: 'Check if the email example@gmail.com exists'"}]})
for el in result["messages"]:
    print(el.content)

## Custom middleware example (logging / debugging hook)

You can define your own middleware by subclassing or using hooks to run custom logic.  
More info about decorators [here](https://docs.langchain.com/oss/python/langchain/middleware/custom#decorator-based-middleware).  

This is useful for logging, analytics, custom validations, formatting, and other custom behaviors.


In [ ]:
from langchain.agents.middleware import ModelRequest, dynamic_prompt


@dynamic_prompt
def context_aware_prompt(request: ModelRequest) -> str:
    """
    Middleware that handles conversation context.
    """
    message_count = len(request.messages)
    print(f"Context Middleware: messages={message_count}")
    prompt_parts = ["You are a helpful assistant."]
    if message_count >= 2:
        prompt_parts.append("This is a long conversation - be extra concise.")
        print(" State Context: Long conversation detected")
    final_prompt = "\n".join(prompt_parts)
    return final_prompt


agent = create_agent(
    model=chat,
    middleware=[context_aware_prompt]
)


res2 = agent.invoke({"messages": [
    {"role": "user", "content": "Give me exampples for the most powerfull quantcopmucters"},
    {"role": "user", "content": "Give me exampples for the most powerfull quantcopmucters"}
]})
for el in res2["messages"]:
    print(el.content)

## Context engineering in agents

### Overview

The hardest part of building reliable agents (or any LLM-powered application) is not just picking a strong model — it’s ensuring the right context is passed to that model for each call. When agents fail in real-world use, it is often because the context was wrong, incomplete, or poorly formatted.

**Context engineering** means deliberately constructing and managing what context (information, state, memory, configuration) is made available to the LLM — and how — so that tasks succeed reliably.  

With LangChain + LangGraph, context is first-class: you get **runtime context**, **state**, and **persistent store**, with full control over how these feed into model calls, tools, prompts, and middleware.  
More info [here](https://docs.langchain.com/oss/python/concepts/context).

Now, let's see how static runtime context works — passing fixed configuration (like role, style, or preferences) to the agent so it influences every model call without changing during execution.


In [ ]:
from dataclasses import dataclass

from langchain.agents import create_agent
from langchain.agents.middleware import dynamic_prompt, ModelRequest

# Define context schema for agent
@dataclass
class ContextSchema:
    communication_style: str  # e.g., "Junior Analyst", "Portfolio Manager"

# Dynamic prompt middleware that adapts to user communication style
@dynamic_prompt
def personalized_prompt(request: ModelRequest) -> str:  
    prompt_parts = ["You are a helpful assistant."]
    # Get communication style from runtime context
    communication_style = request.runtime.context.communication_style

    if communication_style == "Junior Analyst":
        prompt_parts.append(
            "Provide detailed, step-by-step breakdowns and rationales."
        )
    elif communication_style == "Portfolio Manager":
        prompt_parts.append(
            "Provide concise summaries emphasizing decision-critical metrics and outcomes."
        )
    else:
        prompt_parts.append("Provide balanced responses.")
    
    final_prompt = "\n".join(prompt_parts)
    print(f"***Final prompt: {final_prompt}***")
    return final_prompt

tools = [{
    "type": "web_search",
    "function": {}
}]

agent = create_agent(
    model=chat,
    tools=tools,
    middleware=[personalized_prompt],
    context_schema=ContextSchema,
)

res3 = agent.invoke(
    {"messages": [{"role": "user", "content": "Analyze Tesla stock performance in 2024 and explain the key drivers step by step."}]},
    context=ContextSchema(communication_style="Junior Analyst")  
)
for el in res3["messages"]:
    print(f"{el.content[:300]}...")

### Example: Context-aware prompt using persistent store and runtime context

This example shows how to personalize agent behavior using **user-specific preferences** stored in a **persistent store**, based on a unique `user_id`.

The `personalized_prompt` middleware:
- Accesses `user_id` from the runtime context  
- Retrieves user preferences (e.g., communication style) from the store  
- Dynamically adjusts the system prompt based on stored preferences  
- Falls back to a balanced style if no valid preference is found  

This enables:
 - Personalized responses per user  
 - Persistent preference storage across sessions  
 - Automatic adaptation of prompt behavior using runtime context  


In [ ]:
from dataclasses import dataclass


@dataclass
class UserState:
    user_id: str


@dynamic_prompt
def personalized_prompt(request: ModelRequest) -> str:
    """
    Middleware to generate prompts based on stored user preferences.
    """
    store = request.runtime.store  # Access persistent storage
    user_id = str(request.runtime.context.user_id)  # Get current user ID
    user_prefs = store.get(("preferences",), user_id)  # Fetch preferences for user

    prompt_parts = ["You are a helpful assistant."]

    if not user_prefs or not user_prefs.value.get("communication_style"):
        prompt_parts.append(
            "Provide balanced responses"
        )
    elif user_prefs.value.get("communication_style") == "Junior Analyst":
         prompt_parts.append(
                "Provide detailed, step-by-step breakdowns and rationales for all responses."
            )
    elif user_prefs.value.get("communication_style") == "Portfolio Manager":
        prompt_parts.append(
                "Provide concise summaries emphasizing decision-critical metrics and outcomes."
            )
    else:
        prefs = user_prefs.value.get("communication_style")
        print(f"Found unexpected preference - {prefs}")
        prompt_parts.append(
            "Provide balanced responses"
        )
       
    final_prompt = "\n".join(prompt_parts)
    print(f"***Final prompt: {final_prompt}***")
    return final_prompt

### Description — agent with persistent per-user preferences

This code creates a LangChain agent that uses a persistent in-memory store to apply **per-user prompt personalization** at runtime.

**Key behaviors**
- The `personalized_prompt` middleware reads `user_id` from `context` and looks up that user’s preferences in the store to adapt the system prompt (e.g., detailed step-by-step vs. concise summary).
- Using `InMemoryStore` keeps preferences in-process for this example; swap for a persistent store (Redis/DB) in production.

**Notes**
- `user_id` used when calling `agent.invoke(..., context=UserState(user_id=user_id))` controls which stored preferences apply.

In [ ]:
from langchain.agents import create_agent
from langgraph.store.memory import InMemoryStore


agent = create_agent(
    model=chat,
    tools=tools,
    middleware=[personalized_prompt],
    context_schema=UserState,
    store=InMemoryStore()
)

# Reference to the agent's store
store = agent.store

# Example user IDs
user_id = 1

# Store user preferences in the persistent store
store.put(("preferences",), user_id, {"communication_style": "Junior Analyst"})
store.put(("preferences",), 2, {"communication_style": "Portfolio Manager"})


# Invoke the agent for a specific user with runtime context
res3 = agent.invoke(
    {"messages": [{"role": "user", "content": "Analyze Tesla stock performance in 2024 and explain the key drivers step by step."}]},
    context=UserState(user_id=user_id)  # runtime context determines which preferences apply 
)


for el in res3["messages"]:
    print(f"{el.content[:300]}...")

## LangChain + Writer MCP cookbook: Building research agents with streamable HTTP 

This section demonstrates:

- How the Writer MCP client works.
- How to register and use tools.
- Running test queries.
- Extending the research tool.

First, install the required library:

In [ ]:
%pip install langchain-mcp-adapters

## Building a custom MCP client with LangChain

Here, we import the necessary libraries for creating a research agent with LangChain and Writer MCP.

In [ ]:
import asyncio
from langchain_mcp_adapters.client import MultiServerMCPClient
from langchain.agents import create_agent
from langchain_writer import ChatWriter
from langchain.tools import tool

## Creating an asynchronous research task

This code defines an asynchronous task for querying the Writer MCP client. 
- It sets up a system prompt to guide the research assistant.
- Initializes the MCP client with a streamable HTTP transport.
- Loads the ChatWriter model.
- Fetches available tools and creates an agent.
- Streams and prints responses in real time, accumulating the final response.


In [ ]:
async def _async_task(query: str) -> str:
    system_prompt = """
    You are a research assistant with access to Writer documentation.
    When answering questions:
    1. Focus on relevant documentation only.
    2. Provide clear, concise bullet points.
    3. Avoid opinions or unnecessary elaboration.
    """

    client = MultiServerMCPClient(
        {
            "research": {
                "transport": "streamable_http",
                "url": "https://dev.writer.com/mcp",
            }
        }
    )

    model = ChatWriter(model="palmyra-x5")
    tools = await client.get_tools()
    agent = create_agent(model, tools, system_prompt=system_prompt)

    final_response = ""

    # Stream
    async for chunk in agent.astream(
        {"messages": [{"role": "user", "content": query}]},
        stream_mode="updates"
    ):
        for step, data in chunk.items():
            text = data['messages'][-1].content_blocks[0].get('text', "")
            print(f"{text[:100]}...")
            final_response += text

    return f"[Research] Information from internal documentation: {final_response[:800]}..."



## Testing the research

This code tests the `_async_task` function with a sample query:

- Demonstrates how to format the query with tone and style instructions.
- Runs the async task directly in a Jupyter environment.
- Prints the streamed response and the final accumulated result.


In [ ]:

test_query = (
    "I'm interested in no-code apps. "
    "Use a friendly tone and concise bullet points with key–value pairs only."
)

print("🔍 Running _async_task directly for testing...\n")
result = await _async_task(test_query)
print("\n\n✅ Response:\n", result)

## Adding the MCP client as a research tool

In this step, we define a custom tool that integrates the Writer MCP client into our LangChain agent.  

This tool allows the agent to:

- Query Writer documentation and internal resources.
- Provide concise, actionable insights in response to user queries.
- Support research tasks such as product comparisons, feature analysis, and platform evaluation.

The tool is annotated with the `@tool` decorator from LangChain, giving it a name and a description that will be visible to the agent.  

In [ ]:
from langchain.tools import tool

@tool(
    "Research",
    description=(
        "Research documentation, products, or platform comparisons using Writer's MCP client and Palmyra X5 model. "
        "Use this tool when the user asks to compare, analyze, or get detailed insights on products, features, or platforms. "
        "Required: query."
    ),
)
async def writer_research_tool(query: str) -> str:
    """
    Query Writer documentation and return concise, actionable insights.

    Parameters:
        - query (str): The question or topic to research.

    """
    return await _async_task(query)


In th next section, we build a custom customer support agent using LangChain and the Writer MCP client. The `writer_research_tool` will be included as a tool into a new agent

In [ ]:
from langchain.agents import create_agent

def create_customer_support_agent():
    chat = ChatWriter(model="palmyra-x5")

    agent = create_agent(
        model=chat,
        tools=[
            writer_research_tool
        ],
        system_prompt=(
            "You are a customer support assistant for an e-commerce platform. Use tools to handle user requests.\n\n"
            "Tool Usage:\n"
            "[Research] - Research documentation or products using Writer's MCP client and Palmyra X5 model"
            "Rules:\n"
            "- Ask for missing required parameters before calling tools.\n"
        ),
    )
    return agent

# Example query
test_query = "tell me about development platform and give the most popular no-code apps"

agent = create_customer_support_agent()

result = await agent.ainvoke(
    {"messages": [{"role": "user", "content": test_query}]}
)

for el in result["messages"]:
    print(f"{el.type}: {el.content}")

